In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
from os.path import abspath
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys

file_path = abspath('../../outputs/S&P500_1-Year.txt')
sys.stdout = open(file_path, "w")

In [3]:
# Read /data/S&P500.xlsx
sp500 = pd.read_excel(abspath('../../data/S&P/1-Year/S&P500.xlsx'), index_col=0)

sp500.head()

,Open,High,Low,Close*,Adj Close**,Volume
Date,,,,,,
2022-02-09,4547.00,4582.19,4547.00,4576.31,4576.31,1237076000
2022-02-08,4480.02,4531.32,4465.40,4521.54,4521.54,3509330000
2022-02-07,4505.75,4521.86,4471.47,4483.87,4483.87,3291600000
2022-02-04,4482.79,4539.66,4451.50,4500.53,4500.53,3673700000
2022-02-03,4535.41,4542.88,4470.39,4477.44,4477.44,3596830000


In [4]:
# Define the formula for daily return which is (today's closing price - yesterday's closing price) / yesterday's closing price
sp500['daily_return'] = sp500['Close*'].pct_change()

# Define the formula for monthly return which is (this month's closing price - last month's closing price) / last month's closing price
sp500['monthly_return'] = sp500['Close*'].pct_change(12)

# Define the formula for annual return which is (this year's closing price - last year's closing price) / last year's closing price
sp500['annual_return'] = sp500['Close*'].pct_change(252)

# Define 50 day moving average of the closing price
sp500['50_day_moving_average'] = sp500['Close*'].rolling(50).mean()

# Define 200 day moving average of the closing price
sp500['200_day_moving_average'] = sp500['Close*'].rolling(200).mean()

In [5]:
# Find which month has the highest return
print("Highest returning month: " + str(sp500['monthly_return'].idxmax()))

# Determine number of days where closing price is higher than the opening price
sp500['Close>Open'] = (sp500['Close*'] > sp500['Open'])
print("Number of days where closing price is higher than the opening price: " + str(sp500['Close>Open'].sum()))


# Find how many times the index at the close is lower than the open
sp500['Close<Open'] = (sp500['Close*'] < sp500['Open'])
print("Number of times the index at the close is lower than the open: " + str(sp500['Close<Open'].sum()))

sp500['Up&Down'] = sp500['Close*'].diff()

# The number of up days is the days in the Up&Down column which are positive
print("Number of up days: " + str(sp500['Up&Down'][sp500['Up&Down'] > 0].count()))

# The number of down days is the days in the Up&Down column which are negative
print("Number of down days: " + str(sp500['Up&Down'][sp500['Up&Down'] < 0].count()))

In [6]:
import datetime as dt

fridays = {}
mondays = {}

for i in range(len(sp500)):
    if sp500.index[i].isoweekday() == 5:
        fridays[sp500.index[i]] = sp500['Volume'][i]
    elif sp500.index[i].isoweekday() == 1:
        mondays[sp500.index[i]] = sp500['Volume'][i]

# Find the sum of fridays and mondays
print("Sum of fridays: " + str(sum(fridays.values())))
print("Sum of mondays: " + str(sum(mondays.values())))

# Print whether the friday or monday sum is greater
if sum(fridays.values()) > sum(mondays.values()):
    print("Friday sum is greater")
else:
    print("Monday sum is greater")

# # Find any correlation between the volume and return
correlation = sp500['Volume'].corr(sp500['daily_return'])
print("Correlation between the volume and return: " + str(correlation))

In [14]:
# Save the dataframe to the initial file that was opened
sp500.to_csv(abspath('../../data/S&P/1-Year/S&P500.csv'))